In [23]:
# Import Packages
import numpy as np
from tqdm.notebook import tqdm
import ee

In [ ]:
# Authenticate & Initialize Google Earth Engine Account
ee.Authenticate()
ee.Initialize()

In [25]:
# Import geemap Package
import geemap

In [26]:
geemap.ee_search()

In [192]:
# GEEFile(EPSG4326) file in GEE user's assets
fe = ee.FeatureCollection('users/sm0162/GEEFile2022')

In [193]:
# Landsat 7 & 8 Atmospherically Corrected Surface Reflectance Data Image Collections
L7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2');
L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2');
L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2');

In [194]:
#Landsat Masking Function
def mask_landsat(image):
  date = ee.Number.parse(ee.Date(image.get('system:time_start')).format('yyyyMMdd'));
  mask = image.select('QA_PIXEL').bitwiseAnd(ee.Number(2).pow(3).int()).eq(0).And(image.select('QA_PIXEL').bitwiseAnd(ee.Number(2).pow(4).int()).eq(0));
  satellite = ee.String(image.get('SPACECRAFT_ID'));
  image = ee.Image(ee.Algorithms.If(
    satellite.compareTo('LANDSAT_7').eq(0),
    image.select(['SR_B4', 'SR_B7'])
      .multiply(0.0000275).add(-0.2).multiply(10000)
      .rename(['NIR', 'SWIR2']),
    image.select(['SR_B5', 'SR_B7'])
      .multiply(0.0000275).add(-0.2).multiply(10000)
      .rename(['NIR', 'SWIR2'])
    ));
  return image.updateMask(mask.eq(0)).set('date', date);

In [195]:
# Function to generate dNBR
def generateNBR (image):
  return image.normalizedDifference(['SWIR2', 'NIR']).rename('NBR');

In [196]:
# Assign layer name property
LN = 'FireEvents'
fe = fe.map(lambda x: x.set('layerName', LN))

In [202]:
# Main Function
def prepareDNBRGeneration(envelope):
  envelope = ee.Feature(envelope);
  # Which layer?
  layerName = envelope.get('layerName');
  # Get Unique Feature ID
  featID = ee.Number(envelope.get('CLUSTER_ID')).format()
  # return layerName
    
  ###  PRE-FIRE START & END DATE DEFINITION ###
  # If input file is generated through QGIS: Use 'Date'
  # If input file is generated through ArcGISPro: Use 'START_TIME'

  cluster_start_date = ee.Date(envelope.get('START_TIME'));
  pre_fire_start_date = (cluster_start_date.advance(-40, 'day'));
  pre_fire_end_date = (cluster_start_date.advance(-1, 'day'));
  
  ###  POST-FIRE START & END DATE DEFINITION ###
  # If input file is generated through QGIS: Use 'Date'
  # If input file is generated through ArcGISPro: Use 'END_TIME'
    
  cluster_end_date = ee.Date(envelope.get('END_TIME'));
  post_fire_start_date = (cluster_end_date.advance(1, 'day'));
  post_fire_end_date = (cluster_end_date.advance(40, 'day'));

  preImageL7 = L7.filterBounds(envelope.geometry()).filterDate(pre_fire_start_date, pre_fire_end_date);
  preImageL8 = L8.filterBounds(envelope.geometry()).filterDate(pre_fire_start_date, pre_fire_end_date);
  preImageL9 = L9.filterBounds(envelope.geometry()).filterDate(pre_fire_start_date, pre_fire_end_date);
    
  preImageL8L9 = preImageL8.merge(preImageL9)
  
  preImage = ee.Algorithms.If(preImageL8L9.size().eq(0), preImageL7, preImageL8L9);
  preImage = ee.ImageCollection(preImage).map(mask_landsat);
    
  postImageL7 = L7.filterBounds(envelope.geometry()).filterDate(post_fire_start_date, post_fire_end_date);
  postImageL8 = L8.filterBounds(envelope.geometry()).filterDate(post_fire_start_date, post_fire_end_date);
  postImageL9 = L9.filterBounds(envelope.geometry()).filterDate(post_fire_start_date, post_fire_end_date);

  postImageL8L9 = postImageL8.merge(postImageL9)
  
  postImage = ee.Algorithms.If(postImageL8L9.size().eq(0), postImageL7, postImageL8L9);
  postImage = ee.ImageCollection(postImage).map(mask_landsat);


  preImage = preImage.map(generateNBR);
  preImage = preImage.median().multiply(1000).rename([ee.String('preImage_').cat(layerName).cat('_').cat(featID)]);

  postImage = postImage.map(generateNBR);
  postImage = postImage.median().multiply(1000).rename([ee.String('postImage_').cat(layerName).cat('_').cat(featID)]);

  imageName = ee.String("dNBR_").cat(layerName).cat('_').cat(featID)
  dNBR = preImage.subtract(postImage).rename([imageName])
  image = dNBR
  return image.clip(envelope.geometry())

In [203]:
# Run dNBR Generation Function
image = prepareDNBRGeneration(fe.first())
image.bandNames().get(0).getInfo()

'dNBR_FireEvents_1'

In [204]:
# Create list of dNBR events
listfe = fe.toList(fe.size());
listfeNBR = listfe.map(prepareDNBRGeneration)
size = fe.size().getInfo()

In [ ]:
# Google Drive Export Function
# Creates folder "dNBR_Output" in user's Google Drive

for i in tqdm(range(size)):
  i = int(i)
  image = ee.Image(listfeNBR.get(i))
  task_config = {
        'image': image,
        'fileNamePrefix': 'dNBR'+str(image.bandNames().get(0).getInfo().split('_')[-1]),
        'crs': 'EPSG:4326',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'skipEmptyTiles': True,
        'folder': 'dNBR_Output2022_40Days'
    }
  task = ee.batch.Export.image.toDrive(**task_config)
  task.start()

  0%|          | 0/65 [00:00<?, ?it/s]